In [1]:
#General Import 

import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import pandas as pd 
import random
import math
import time
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime
import operator 
plt.style.use('seaborn')
%matplotlib inline

In [2]:
#Import CSVs

#Import the data
import pandas as pd
confirmed_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
deaths_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv", error_bad_lines=False)
recoveries_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv',  error_bad_lines=False)
#latest_cases = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-04-2020.csv")
#confirmed_df_alternativ = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")



In [3]:
#Import Population 

population_df = pd.read_csv("Population.csv", encoding = "ISO-8859-1") #World Population from UN

# Overview

First we create a static table to confirm the correct data import and get an overview


In [4]:


#Funcion to add deaths and recoveries to global dataframe, merging numbers either to province or country

import warnings
warnings.filterwarnings("ignore")

#input: global data frame
global_df_alt = confirmed_df_alternativ.iloc[:,[0,1,-1]] 
global_df_alt.columns = [*global_df_alt.columns[:-1], 'Confirmed']
dataframe = global_df_alt
add_column = "Number of Deaths"

#to merge: recoveries and death
def merge_column(dataframe, merge, add_column):
    merge = merge.iloc[:,[0,1,-1]]
    values = []
    for x, y in zip(dataframe["Province/State"], dataframe["Country/Region"]):
        if pd.isnull(x):
            if y in merge["Country/Region"].tolist():
                values.append((merge[merge["Country/Region"]== y].iloc[0,-1]))
            else:
                values.append(np.nan)
        else:
            if x in merge["Province/State"].tolist():
                values.append(merge[merge["Province/State"]== x].iloc[0,-1])
            else:
                values.append(np.nan)
    values
    dataframe[add_column] = values
    #dataframe[add_column] = global_df_alt[add_column].astype('Int64')
    return(dataframe)

#Add Columns "Deaths", "Recoveries" and calculate fields "Number of active Cases" and "Mortality Rate"

global_df_alt = merge_column(global_df_alt, deaths_df, "Number of Deaths")
global_df_alt = merge_column(global_df_alt, recoveries_df, "Number of Recoveries")
global_df_alt["Number of Active Cases"] = global_df_alt["Confirmed"]-global_df_alt["Number of Deaths"]-global_df_alt["Number of Recoveries"]
global_df_alt.rename(columns={"Country/Region": "Country Name", "Confirmed": "Number of Confirmed Cases"},inplace=True)
global_df_alt["Mortality Rate"] =  global_df_alt["Number of Deaths"] / global_df_alt["Number of Confirmed Cases"] 

#ONe global Database with all regions, and one grouped by country
global_regions = global_df_alt
global_countries = global_df_alt.groupby(["Country Name"]).sum().sort_values(by=["Number of Confirmed Cases", "Country Name"], ascending=[False, True])

global_countries.style.background_gradient(cmap='Greens')

NameError: name 'confirmed_df_alternativ' is not defined

# Table for Tableau Visualisation

Here I build a pivot table, having only one value columns.

Columns:

Country/Region - Level of Detail - Province/State - Category - Date - Numbers

In [16]:


#Dropping Latitude and Longitude and add Category as column
recoveries = recoveries_df.drop(["Lat", "Long"], axis = 1)
recoveries.insert(0, "Category", "Recoveries")
confirmed = confirmed_df.drop(["Lat", "Long"], axis = 1)
confirmed.insert(0, "Category", "Confirmed")
deaths = deaths_df.drop(["Lat", "Long"], axis = 1)
deaths.insert(0, "Category", "Deaths")


#Join all imported tables
timeline = pd.concat([confirmed, deaths, recoveries], sort=False).reset_index()
timeline = timeline.drop("index", axis = 1)

#Add Level of Detail (Country: 1, Province, State: 2, City: 3)
timeline["Level of Detail"] = np.nan

for i in timeline.index:
    if pd.isnull(timeline.loc[i, "Province/State"]):
        timeline.loc[i, "Level of Detail"] = 1
    else: 
        timeline.loc[i, "Level of Detail"] = 2
timeline = timeline.astype({"Level of Detail": int}) 

timeline_1 = timeline

In [6]:
#Check, if Country (Level of Detail = 1) exists. For example, for Australia, Canada and China it doesnot.

#If it exists, we do not add a total. If not, we have to add a total over all States/Provinces
timeline_order = timeline.copy()

list_countries = timeline["Country/Region"].unique()
list_categories= timeline["Category"].unique()
test = timeline.groupby(["Country/Region","Level of Detail","Category"]).sum().reset_index().rename(columns={0:'count'})

country_counted =[]
country_not_counted=[]
for j in list_categories:
    for i in list_countries:
        if 1 in  test[(test["Country/Region"]==i) & (test["Category"]== j)]["Level of Detail"].tolist():
            test = test.drop(test[(test["Country/Region"]==i) & (test["Category"]== j)]["Level of Detail"].index)
        else:
            country_not_counted.append(i)

test["Level of Detail"] = 1
test["Province/State"] = np.nan

timeline_hz = timeline.append(test)[test.columns.tolist()]

#Reordering the DataFrame, it is conveniont to have the categories on the left side 

timeline_hz = timeline_hz.reset_index()
timeline_hz = timeline_hz.drop("index", axis = 1)
col_name="Province/State"
first_col = timeline_hz.pop(col_name)
timeline_hz.insert(2, col_name, first_col) 
saver = timeline_hz

timeline_hz
timeline2 = timeline_hz

In [7]:
#Add Global Numbers as it was a Country

timeline_hz = saver.copy()
Categories = timeline_hz["Category"].unique()

for i in Categories:
    DF_Countries = timeline_hz[(timeline_hz["Level of Detail"] == 1)&   (timeline_hz["Category"]==i)]
    add_list=[]
    add_list.append("Global")
    add_list.append(1)
    add_list.append(None)
    add_list.append(i)
    for d in DF_Countries.columns[4:]:
        add_list.append(DF_Countries[d].sum())

    timeline_hz.loc[(len(timeline_hz))] = add_list

timeline_3 = timeline_hz

In [8]:
#Add Daily Rates as category 
#- after finishing the Tableau Dashboard, I realized that it would have been better to create a new column, adding the rate to each row. 
#Maybe here it will be better to build a function. Version Now is a growth rate about all Categories!


Categories = timeline_hz["Category"].unique() 
#Categories = ["Confirmed", "Deaths", "Recoveries"]
add_list = []
columns_copy = ["Country/Region", "Level of Detail"]

#to optimize: how "in" for with list selected columns??
timeline_hz.reset_index()
index = timeline_hz.index
for i in index:
    add_list = []
    list_root = timeline_hz.iloc[i,:].tolist()
    add_list.append(list_root[0])
    add_list.append(list_root[1])
    add_list.append(list_root[2])
    add_list.append(list_root[3] + " Daily Growth")
    for d in range(4,len(timeline_hz.columns)):
        if d == 4:
            add_list.append(list_root[4])
        else:
            add_list.append(list_root[d]- list_root[(d-1)])
    
    timeline_hz.loc[(len(timeline_hz)+1)] = add_list

timeline_4 = timeline_hz

In [9]:
#Bring timeline from horizontal to vertical order

id_vars = timeline_hz.columns[[0,1, 2,3]]
value_vars=timeline_hz.columns[4:]
var_name = "Date"
Value_name = "Numbers"
timeline_vert = pd.melt(timeline_hz,id_vars=id_vars,
                        value_vars=value_vars,
                              var_name=var_name,value_name=Value_name)

In [10]:
#Save CSV

timeline_vert.to_csv(r'timline_vert.csv', index = False)

# Maybe add Population to csv

The CSV for Tableau is saved. From here on it is not finished yet

In [12]:
#Switch categories to columns -> Not recomanndable for Tableau

categories = timeline_vert["Category"].unique()
test = pd.concat([timeline_vert.set_index(["Country/Region", "Level of Detail",
                             "Date", "Province/State"]).groupby('Category')['Numbers'].get_group(key) for key in categories],axis=1)

test.columns=categories
test.reset_index(inplace=True)



tableau = test

tableau

,Country/Region,Level of Detail,Date,Province/State,Confirmed,Deaths,Recoveries,Confirmed Daily Growth,Deaths Daily Growth,Recoveries Daily Growth
0,Afghanistan,1,1/22/20,NaN,0,0,0.0,0,0,0.0
1,Afghanistan,1,1/23/20,NaN,0,0,0.0,0,0,0.0
2,Afghanistan,1,1/24/20,NaN,0,0,0.0,0,0,0.0
3,Afghanistan,1,1/25/20,NaN,0,0,0.0,0,0,0.0
4,Afghanistan,1,1/26/20,NaN,0,0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...
24115,Zimbabwe,1,4/5/20,NaN,9,1,0.0,0,0,0.0
24116,Zimbabwe,1,4/6/20,NaN,10,1,0.0,1,0,0.0
24117,Zimbabwe,1,4/7/20,NaN,11,2,0.0,1,1,0.0
24118,Zimbabwe,1,4/8/20,NaN,11,3,0.0,0,1,0.0


In [14]:
#The aim is to have everything in one big dataframe. We add the column "Population". Hence we are dealing with countries
#and states, we must import several databases (world, states per country (USA, GERMANY, etc))
#We face the problem of having different Countrynames (US = United States of America)


population = population_df.copy()
population.drop("ISO 3166-1 numeric code", axis = 1, inplace=True)
population.drop("Note", axis = 1, inplace=True)
population.rename(columns={"Location": "Country Name", "2020": "Population"},inplace=True)
population["Country Name"] = population["Country Name"].str.strip()
population["Population"] = population["Population"].str.replace(' ', '')
#global_df_alt.join(population, lsuffix = "Country Name", rsuffix = "Location", how = "left")
#Here i have to build a string finder!
name_org = ["United States of America","Russian Federation", "Dem. People's Republic of Korea","Iran (Islamic Republic of)"]
name_change = ["US", "Russia", "Korea, South", "Iran"]

population = population.replace(name_org, name_change)
population["Population"] = population["Population"].astype(int)

#Instead of Join/Merge function, I write a function 

In [15]:
#Add population column as far as disposed and calculate cases per 100.00 habitans

global_countries = pd.merge(global_countries, population, on='Country Name', how= "left")
global_regions = pd.merge(global_regions, population, on='Country Name', how= "left")

global_regions_pop = global_regions.drop(global_countries[global_countries["Population"].isnull()].index) #countries without population
global_regions_pop["Cases per 100.000 inhabitans"] = global_regions_pop["Number of Confirmed Cases"] / (
                                                    global_regions_pop["Population"]) * 100

global_countries_pop = global_countries.drop(global_countries[global_countries["Population"].isnull()].index) #countries without population
global_countries_pop["Cases per 100.000 inhabitans"] = global_countries_pop["Number of Confirmed Cases"] / (
                                                    global_countries_pop["Population"]) * 100

NameError: name 'global_countries' is not defined

In [ ]:
global_countries = global_countries_pop.sort_values(by = "Number of Confirmed Cases", ascending = False)


global_countries.style.background_gradient(cmap='Greens')
